In [2]:
from pydantic import BaseModel

In [3]:
class Car(BaseModel):
    brand:str
    price:int
    country:str
    
car = Car(brand='BMW',price=200000000,country='India')

In [4]:
print(car)

brand='BMW' price=200000000 country='India'


In [6]:
type(car)

__main__.Car

In [8]:
from dataclasses import dataclass

@dataclass
class Room():
    names:str
    no_of_people:int
    city:str
    
room = Room(names="sai",no_of_people=4,city='Hyderabad')
print(room)
    


Room(names='sai', no_of_people=4, city='Hyderabad')


## 1. Purpose & Core Functionality
#### Standard dataclass (from dataclasses in stdlib):
- Meant for lightweight data containers—automatically generates __init__, __repr__, __eq__, etc.
- No runtime type validation—type hints are for developer reference and static checkers only.

#### Pydantic:
- Provides runtime validation, automatic type coercion, JSON serialization, schema generation, error reporting, and nested model   handling—ideal for robust data processing, especially in web APIs.

## 2. Validation & Type Coercion
#### Dataclasses:
- Don’t validate types by default. You must write manual checks inside __post_init__, which can be verbose and error-prone.

#### Pydantic:
- Automatically converts types on instantiation (e.g., "42" → 42) and raises detailed validation errors. Supports declarative validation via @validator, and constraints like gt, le, multiple_of, etc.

## 3. Performance & Dependencies
#### Dataclasses:
- Lightweight, fast, and part of the standard library—great for simple, internal data structures.

#### Pydantic:
- Heavier due to validation logic—slower instantiation, more memory usage, and adds an external dependency. Ideal when data correctness is important, like in APIs or file I/O.

## 4. Hybrid Options: pydantic.dataclasses.dataclass vs pydantic.BaseModel
#### pydantic.dataclasses.dataclass decorator:
- Acts like a standard dataclass but adds optional validation. More lightweight than BaseModel.

#### pydantic.BaseModel:
- Full-featured model with JSON export (.dict(), .json()), schema generation, validate_assignment, aliasing, and more powerful capabilities—but also more overhead.

## 5. Developer Community Perspective
#### Many developers note:
- Pydantic models are significantly more “heavyweight” than standard dataclasses—they serve mainly as validators.
- Pydantic doesn’t make Python more strictly typed—it’s a library to handle complex serialization and validation.

# Note:
- Use @dataclass for minimal overhead and clean static containers without runtime checks.
- Use Pydantic (BaseModel) when data correctness, JSON I/O, and validation are essential.
- Use pydantic.dataclasses.dataclass for a hybrid approach—structured models with optional validation.

In [11]:
from typing import Optional

class Employee(BaseModel):
    id:int
    name:str
    department:str
    salary: Optional[float] = None
    is_active: Optional[bool] = True

In [12]:
emp1 = Employee(id=1,name="john",department='IT')
print(emp1)

id=1 name='john' department='IT' salary=None is_active=True


In [13]:
emp2 = Employee(id=2,name="Wick",department='Hr',salary=30000,is_active=False)
print(emp2)

id=2 name='Wick' department='Hr' salary=30000.0 is_active=False


#### Definition:
- Optional[type]: Indicates the field can be none.
- Default value (=None or =True): make the field optional
- Required fields must still be provided
- Pydantic validates types even for optional fields when values are provided.

In [16]:
from pydantic import BaseModel
from typing import List

class Classroom(BaseModel):
    room_number:str
    students: List[str]
    capacity: int

In [17]:
classroom = Classroom(
    room_number='302',
    students=['sai','vijay','vikram'],
    capacity=30
)

print(classroom)

room_number='302' students=['sai', 'vijay', 'vikram'] capacity=30


In [18]:
from pydantic import BaseModel
from typing import Optional
from datetime import datetime

class User(BaseModel):
    id: int
    name: str
    email: str
    age: Optional[int] = None
    is_active: bool = True
    created_at: datetime = datetime.now()

# Usage
user = User(id=1, name="Alice", email="alice@example.com", age=30)
print(user.name) 
print(user.model_dump())  

Alice
{'id': 1, 'name': 'Alice', 'email': 'alice@example.com', 'age': 30, 'is_active': True, 'created_at': datetime.datetime(2025, 6, 20, 16, 37, 11, 14814)}


In [21]:
from pydantic import BaseModel, field_validator, Field
import re

class UserProfile(BaseModel):
    username: str = Field(..., min_length=3, max_length=20)
    email: str
    password: str = Field(..., min_length=8)
    age: int = Field(..., ge=13, le=120)
    
    @field_validator('email')
    def validate_email(cls, v):
        if not re.match(r'^[^@]+@[^@]+\.[^@]+$', v):
            raise ValueError('Invalid email format')
        return v
    
    @field_validator('username')
    def validate_username(cls, v):
        if not v.isalnum():
            raise ValueError('Username must be alphanumeric')
        return v

# This will raise validation errors
try:
    user = UserProfile(
        username="ab",  
        email="invalid-email",
        password="123",  
        age=200  
    )
except ValueError as e:
    print(e)

4 validation errors for UserProfile
username
  String should have at least 3 characters [type=string_too_short, input_value='ab', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_too_short
email
  Value error, Invalid email format [type=value_error, input_value='invalid-email', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error
password
  String should have at least 8 characters [type=string_too_short, input_value='123', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_too_short
age
  Input should be less than or equal to 120 [type=less_than_equal, input_value=200, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal


In [22]:
from typing import List

class Address(BaseModel):
    street: str
    city: str
    country: str
    zip_code: str

class Company(BaseModel):
    name: str
    address: Address
    employees: List[str]

class Employee(BaseModel):
    name: str
    position: str
    salary: float
    company: Company
    home_address: Optional[Address] = None

# Usage with nested data
company_data = {
    "name": "Tech Corp",
    "address": {
        "street": "123 Tech St",
        "city": "San Francisco",
        "country": "USA",
        "zip_code": "94105"
    },
    "employees": ["Alice", "Bob", "Charlie"]
}

employee = Employee(
    name="Alice",
    position="Developer",
    salary=75000,
    company=company_data
)